<h1 style="text-align:center;">Reporte de futuros y tasas</h1>

## Tasas implicitas de Futuros

In [1]:

import pandas as pd
from datetime import date, timedelta
from futures_rfx import *
from bcra import *
from IPython.display import display, HTML
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

end = date.today()
start = end - timedelta(days=30)

df = get_futures(start,end)
TAMAR_SERIE = get_tamar_serie(start, end ,id_variable=44)
bills = get_bills_rates()
rates = get_implied_rates(df)

volume = df.pivot(index='dateTime',columns='symbol',values=['volume'])
volume.columns = volume.columns.droplevel(0)  # Drop the top level of the MultiIndex

openInterest = df.pivot(index='dateTime',columns='symbol',values=['openInterest'])
openInterest.columns = openInterest.columns.droplevel(0)  # Drop the top level of the MultiIndex
last_run = date.today().strftime('%Y-%m-%d')
print('Ultima actualización de datos:', last_run)

Ultima actualización de datos: 2025-10-27


In [2]:
# Plot limits
last_futures_close = get_last_futures_close_fx_limits(df)
plot_limits_plotly(last_futures_close)
# Plot rates
plot_smooth_plotly(rates[rates < 100])
# Table display
rates_display = rates.copy()
rates_display = rates_display/100
rates_display.index = rates_display.index.strftime("%Y-%m-%d")
rates_display = rates_display.style.background_gradient(cmap="YlOrRd")
rates_display = rates_display.format("{:.1%}")
display(HTML(rates_display.to_html().replace('<table',
                                '<table style="font-size:10px"')))

plot_tamar_serie_plotly(TAMAR_SERIE/100)
plot_tamar_serie_plotly(bills[['expiration_date', 'tna']].set_index('expiration_date'), 'TNA Letras del Tesoro a Tasa Fija', 'Fecha de vencimiento', 'TNA %')


symbol,DLR012026,DLR022026,DLR052026,DLR102025,DLR112025,DLR122025
dateTime,,,,,,
2025-09-29,57.3%,nan%,44.6%,76.6%,71.0%,64.2%
2025-09-30,62.7%,56.6%,46.4%,109.8%,82.4%,70.3%
2025-10-01,46.6%,nan%,37.0%,62.1%,57.1%,50.6%
2025-10-02,41.3%,38.7%,33.8%,30.7%,43.9%,43.5%
2025-10-03,38.8%,36.4%,33.4%,25.8%,41.1%,40.2%
2025-10-06,nan%,41.4%,35.9%,39.1%,51.9%,48.1%
2025-10-07,48.0%,nan%,37.4%,42.3%,57.3%,51.9%
2025-10-08,47.1%,43.2%,37.3%,44.1%,59.2%,50.7%
2025-10-09,37.9%,35.4%,32.9%,-7.1%,36.6%,38.7%


## Interes Abierto y Volumen diario


In [3]:

plot_futures_plotly(data=volume, kind='bar', ylabel="Volume", title="Volume Over Time", legend_name = "Volume")
plot_futures_plotly(data=openInterest, kind='bar', ylabel="Open Interest", title="Open Interest Over Time", legend_name = "Open Interest")

In [4]:
!py -3.10 -m nbconvert --no-input --to html index.ipynb -output index.html

[NbConvertApp] WARNING | Unrecognized alias: 'output', it will have no effect.
[NbConvertApp] Converting notebook index.ipynb to html
C:\Users\jljuncos\AppData\Roaming\Python\share\jupyter\nbconvert\templates\base\display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'text/vnd.mermaid' -%}
[NbConvertApp] Writing 288039 bytes to index.html
